<a href="https://colab.research.google.com/github/abSETEd/interface_planilha/blob/main/check_phone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas bs4 pyexcel_ods3 openpyxl requests
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import openpyxl  # Para XLSX
import pyexcel_ods3  # Para ODS
from google.colab import files  # Para o upload
import io  # Para processar o arquivo enviado
import re  # Para expressões regulares

# Função para carregar a planilha em diferentes formatos
def carregar_planilha(arquivo_carregado, nome_arquivo):
    try:
        if nome_arquivo.endswith('.csv'):
            df = pd.read_csv(io.BytesIO(arquivo_carregado))
        elif nome_arquivo.endswith('.xlsx'):
            df = pd.read_excel(io.BytesIO(arquivo_carregado), engine='openpyxl')
        elif nome_arquivo.endswith('.ods'):
            data = pyexcel_ods3.get_data(io.BytesIO(arquivo_carregado))
            df = pd.DataFrame(data[list(data.keys())[0]][1:], columns=data[list(data.keys())[0]][0])
        else:
            raise ValueError("Formato de arquivo não suportado. Use CSV, XLSX ou ODS.")

        if 'Site' not in df.columns:
            raise ValueError("A planilha precisa ter uma coluna chamada 'Site'.")

        # Formata a coluna 'Site' adicionando 'https://' se necessário
        df['Site'] = df['Site'].apply(lambda x: f"https://{x}" if not str(x).startswith(('http://', 'https://')) else x)

        return df
    except Exception as e:
        print(f"Erro ao carregar a planilha: {e}")
        return None

# Função para consultar um site e extrair telefones
def consultar_site(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        resposta = requests.get(url, headers=headers, timeout=10)
        resposta.raise_for_status()

        soup = BeautifulSoup(resposta.text, 'html.parser')
        texto = soup.get_text()

        # Padrão flexível pra telefones no Brasil
        padrao_telefone = r'(?:(?:\+?55\s?)?(?:\(?\d{2}\)?)?\s?\d{4,5}[-.\s]?\d{4})'
        telefones = re.findall(padrao_telefone, texto)
        telefones_unicos = list(set(telefones))  # Remove duplicatas

        if telefones_unicos:
            return ', '.join(telefones_unicos)  # Junta os telefones com vírgula
        else:
            return "Nenhum telefone encontrado"
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return "Erro na consulta"

# Função principal
def main():
    print("Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):")

    # Interface de upload
    uploaded = files.upload()  # Abre o seletor de arquivos

    if not uploaded:
        print("Nenhum arquivo foi enviado. Encerrando.")
        return

    # Pega o nome e conteúdo do arquivo enviado
    nome_arquivo = list(uploaded.keys())[0]
    arquivo_carregado = uploaded[nome_arquivo]

    # Carrega a planilha e formata URLs
    df_urls = carregar_planilha(arquivo_carregado, nome_arquivo)
    if df_urls is None:
        return

    print(f"Carreguei {len(df_urls)} linhas da planilha.")

    # Lista para armazenar resultados
    resultados = []

    # Itera pelos sites
    for index, row in df_urls.iterrows():
        url = row['Site']  # Já formatado com 'https://'
        print(f"Consultando: {url}")

        resultado = consultar_site(url)
        resultados.append({'Site': url, 'Telefone': resultado})  # Coluna 'Telefone' no lugar de 'Resultado'
        print(f"Telefones para {url}: {resultado}")

        time.sleep(2)

    # Salva os resultados em CSV
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv('resultados_telefones.csv', index=False)
    print("Resultados salvos em 'resultados_telefones.csv'. Você pode baixar o arquivo na aba 'Files'!")

# Executa o programa
if __name__ == "__main__":
    !pip install pyexcel-ods3  # Instala suporte pra ODS
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.8 MB/s eta 0:00:00
Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):


In [ ]:
!pip install pandas bs4 pyexcel_ods3 openpyxl requests
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import openpyxl  # Para XLSX
import pyexcel_ods3  # Para ODS
from google.colab import files  # Para o upload
import io  # Para processar o arquivo enviado

# Função para carregar a planilha em diferentes formatos
def carregar_planilha(arquivo_carregado, nome_arquivo):
    try:
        if nome_arquivo.endswith('.csv'):
            df = pd.read_csv(io.BytesIO(arquivo_carregado))
        elif nome_arquivo.endswith('.xlsx'):
            df = pd.read_excel(io.BytesIO(arquivo_carregado), engine='openpyxl')
        elif nome_arquivo.endswith('.ods'):
            data = pyexcel_ods3.get_data(io.BytesIO(arquivo_carregado))
            df = pd.DataFrame(data[list(data.keys())[0]][1:], columns=data[list(data.keys())[0]][0])
        else:
            raise ValueError("Formato de arquivo não suportado. Use CSV, XLSX ou ODS.")

        if 'Site' not in df.columns:
            raise ValueError("A planilha precisa ter uma coluna chamada 'Site'.")

        return df
    except Exception as e:
        print(f"Erro ao carregar a planilha: {e}")
        return None

# Função para consultar um site (exemplo básico)
def consultar_site(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        resposta = requests.get(url, headers=headers, timeout=10)
        resposta.raise_for_status()

        soup = BeautifulSoup(resposta.text, 'html.parser')
        texto = soup.get_text()

        return texto.strip()[:100]  # Limita a 100 caracteres pra teste
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return "Erro na consulta"

# Função principal
def main():
    print("Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):")

    # Interface de upload
    uploaded = files.upload()  # Abre o seletor de arquivos

    if not uploaded:
        print("Nenhum arquivo foi enviado. Encerrando.")
        return

    # Pega o nome e conteúdo do arquivo enviado
    nome_arquivo = list(uploaded.keys())[0]
    arquivo_carregado = uploaded[nome_arquivo]

    # Carrega a planilha
    df_urls = carregar_planilha(arquivo_carregado, nome_arquivo)
    if df_urls is None:
        return

    print(f"Carreguei {len(df_urls)} linhas da planilha.")

    # Lista para armazenar resultados
    resultados = []

    # Itera pelos sites
    for index, row in df_urls.iterrows():
        url = row['Site']  # Usa a coluna 'Site' com 'S' maiúsculo
        print(f"Consultando: {url}")

        resultado = consultar_site(url)
        resultados.append({'Site': url, 'Resultado': resultado})
        print(f"Resultado para {url}: {resultado[:50]}...")

        time.sleep(2)

    # Salva os resultados em CSV
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv('resultados_consulta.csv', index=False)
    print("Resultados salvos em 'resultados_consulta.csv'. Você pode baixar o arquivo na aba 'Files'!")

# Executa o programa
if __name__ == "__main__":
    !pip install pyexcel-ods3  # Instala suporte pra ODS
    main()

Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):


Saving Domínios Potencial- planilha teste.xlsx to Domínios Potencial- planilha teste (3).xlsx
Carreguei 200 linhas da planilha.
Consultando: DOMINALOG.COM.BR
Erro ao acessar DOMINALOG.COM.BR: Invalid URL 'DOMINALOG.COM.BR': No scheme supplied. Perhaps you meant https://DOMINALOG.COM.BR?
Resultado para DOMINALOG.COM.BR: Erro na consulta...
Consultando: BRCONDOS.COM.BR
Erro ao acessar BRCONDOS.COM.BR: Invalid URL 'BRCONDOS.COM.BR': No scheme supplied. Perhaps you meant https://BRCONDOS.COM.BR?
Resultado para BRCONDOS.COM.BR: Erro na consulta...
Consultando: RENAPSI.ORG.BR
Erro ao acessar RENAPSI.ORG.BR: Invalid URL 'RENAPSI.ORG.BR': No scheme supplied. Perhaps you meant https://RENAPSI.ORG.BR?
Resultado para RENAPSI.ORG.BR: Erro na consulta...
Consultando: GRUPOCENE.COM.BR
Erro ao acessar GRUPOCENE.COM.BR: Invalid URL 'GRUPOCENE.COM.BR': No scheme supplied. Perhaps you meant https://GRUPOCENE.COM.BR?
Resultado para GRUPOCENE.COM.BR: Erro na consulta...
Consultando: VAENGENHARIA.COM.BR
Er

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import openpyxl  # Para XLSX
import pyexcel_ods3  # Para ODS
from google.colab import files  # Para o upload
import io  # Para processar o arquivo enviado

# Função para carregar a planilha em diferentes formatos
def carregar_planilha(arquivo_carregado, nome_arquivo):
    try:
        if nome_arquivo.endswith('.csv'):
            df = pd.read_csv(io.BytesIO(arquivo_carregado))
        elif nome_arquivo.endswith('.xlsx'):
            df = pd.read_excel(io.BytesIO(arquivo_carregado), engine='openpyxl')
        elif nome_arquivo.endswith('.ods'):
            data = pyexcel_ods3.get_data(io.BytesIO(arquivo_carregado))
            df = pd.DataFrame(data[list(data.keys())[0]][1:], columns=data[list(data.keys())[0]][0])
        else:
            raise ValueError("Formato de arquivo não suportado. Use CSV, XLSX ou ODS.")

        if 'Site' not in df.columns:
            raise ValueError("A planilha precisa ter uma coluna chamada 'Site'.")

        # Formata a coluna 'Site' adicionando 'https://' se necessário
        df['Site'] = df['Site'].apply(lambda x: f"https://{x}" if not str(x).startswith(('http://', 'https://')) else x)

        return df
    except Exception as e:
        print(f"Erro ao carregar a planilha: {e}")
        return None

# Função para consultar um site (exemplo básico)
def consultar_site(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        resposta = requests.get(url, headers=headers, timeout=10)
        resposta.raise_for_status()

        soup = BeautifulSoup(resposta.text, 'html.parser')
        texto = soup.get_text()

        return texto.strip()[:100]  # Limita a 100 caracteres pra teste
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return "Erro na consulta"

# Função principal
def main():
    print("Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):")

    # Interface de upload
    uploaded = files.upload()  # Abre o seletor de arquivos

    if not uploaded:
        print("Nenhum arquivo foi enviado. Encerrando.")
        return

    # Pega o nome e conteúdo do arquivo enviado
    nome_arquivo = list(uploaded.keys())[0]
    arquivo_carregado = uploaded[nome_arquivo]

    # Carrega a planilha e formata URLs
    df_urls = carregar_planilha(arquivo_carregado, nome_arquivo)
    if df_urls is None:
        return

    print(f"Carreguei {len(df_urls)} linhas da planilha.")

    # Lista para armazenar resultados
    resultados = []

    # Itera pelos sites
    for index, row in df_urls.iterrows():
        url = row['Site']  # Já formatado com 'https://'
        print(f"Consultando: {url}")

        resultado = consultar_site(url)
        resultados.append({'Site': url, 'Resultado': resultado})
        print(f"Resultado para {url}: {resultado[:50]}...")

        time.sleep(2)

    # Salva os resultados em CSV
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv('resultados_consulta.csv', index=False)
    print("Resultados salvos em 'resultados_consulta.csv'. Você pode baixar o arquivo na aba 'Files'!")

# Executa o programa
if __name__ == "__main__":
    !pip install pyexcel-ods3  # Instala suporte pra ODS
    main()

Por favor, faça o upload da sua planilha (CSV, XLSX ou ODS):


Saving Domínios Potencial- planilha teste 35.ods to Domínios Potencial- planilha teste 35.ods
Carreguei 36 linhas da planilha.
Consultando: https://DOMINALOG.COM.BR
Erro ao acessar https://DOMINALOG.COM.BR: 406 Client Error: Not Acceptable for url: https://dominalog.com.br/
Resultado para https://DOMINALOG.COM.BR: Erro na consulta...
Consultando: https://BRCONDOS.COM.BR
Resultado para https://BRCONDOS.COM.BR: BRCondos » Gestão Condominial Completa











...
Consultando: https://RENAPSI.ORG.BR
Resultado para https://RENAPSI.ORG.BR: Home - Demà Jovem by Renapsi Renapsi













...
Consultando: https://GRUPOCENE.COM.BR
Resultado para https://GRUPOCENE.COM.BR: Grupo Cene - Medicina e Segurança do Trabalho




...
Consultando: https://VAENGENHARIA.COM.BR
Resultado para https://VAENGENHARIA.COM.BR: Grupo VA Soluções Sustentáveis



















...
Consultando: https://TIGRE.COM
Erro ao acessar https://TIGRE.COM: HTTPSConnectionPool(host='tigre.com', port=443): Max retries excee